This is the second of two parts time-saving quickstart baseline. Submission accuracy is 0.88 in public dataset. In this notebook, there are three points which may benefit you. 
* The nearest interpolation of load_img method is consistent with [part I training](https://www.kaggle.com/electro/keras-baseline-0-88-quickstart-i-training-saving).
* A for loop to predict a batch instead of one image input at a time.
* Generation of submission file by appending batch predictions.

In [ ]:
import numpy as np
import pandas as pd
import os
import tensorflow as tf


Cassava_dir = "../input/cassava-leaf-disease-classification/"

In [ ]:
from tensorflow import keras
model = keras.models.load_model('../input/cassava-baseline-weights/best_weights.h5')

In [ ]:
sub = pd.DataFrame(columns=['image_id','label'])

In [ ]:
def predict_on_batch(test_list, wpath=Cassava_dir, target_size=(380,380)):
    input_batch=[]
    for IMAGE_ID in test_list:
        image = tf.keras.preprocessing.image.load_img(os.path.join(wpath, "test_images",IMAGE_ID), 
                                                      grayscale=False, 
                                                      color_mode="rgb", 
                                                      target_size=target_size, 
                                                      interpolation="nearest")
        input_arr = keras.preprocessing.image.img_to_array(image)
        input_batch.append(input_arr)
    return np.array(input_batch)

In [ ]:
import math

TEST_DIR = '../input/cassava-leaf-disease-classification/test_images/'
test_images = os.listdir(TEST_DIR)

N = len(test_images)
if N == 1:
    BATCH_SIZE = 1
else:
    BATCH_SIZE = 16

In [ ]:
for batch_index in range(math.ceil(N/BATCH_SIZE)):
    if batch_index*BATCH_SIZE+BATCH_SIZE < N:
        test_X = predict_on_batch(test_images[batch_index*BATCH_SIZE:batch_index*BATCH_SIZE+BATCH_SIZE])
        predictions = model.predict(test_X).argmax(axis = 1)
        sub_batch = pd.DataFrame({'image_id':test_images[batch_index*BATCH_SIZE:batch_index*BATCH_SIZE+BATCH_SIZE],
                    'label':list(predictions)})
    else:
        test_X = predict_on_batch(test_images[batch_index*BATCH_SIZE:])
        predictions = model.predict(test_X).argmax(axis = 1)
        sub_batch = pd.DataFrame({'image_id':test_images[batch_index*BATCH_SIZE:],
                    'label':list(predictions)})
        
    sub = sub.append(sub_batch, ignore_index=True)

In [ ]:
sub['label'] = sub['label'].astype('int64')
sub

In [ ]:
sub.to_csv('submission.csv', index = False)